<a target="_blank" href="https://colab.research.google.com/github/UpstageAI/cookbook/blob/main/Solar-Fullstack-LLM-101/98_2_edu_application.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Solar Application Examples
This file provides simple example codes that can be created using Solar.

## Preparation
Get a Solar API key in advance.

In [1]:
! pip3 install -qU langchain langchain-upstage langchain_community python-dotenv firecrawl-py==0.0.20 marko

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-experimental 0.0.65 requires langchain-community<0.3.0,>=0.2.16, but you have langchain-community 0.3.0 which is incompatible.
langchain-experimental 0.0.65 requires langchain-core<0.3.0,>=0.2.38, but you have langchain-core 0.3.5 which is incompatible.
ragas 0.1.20 requires langchain-core<0.3, but you have langchain-core 0.3.5 which is incompatible.
guardrails-ai 0.5.10 requires langchain-core<0.3,>=0.1, but you have langchain-core 0.3.5 which is incompatible.


## UPSTAGE_API_KEY
To obtain your Upstage API key, follow these steps:

1. Visit the Upstage AI console at <https://console.upstage.ai>.
2. Sign up for an account if you don't already have one.
3. Log in to your account.
4. Navigate to the API key section.
5. Generate your API key.
6. Copy the key and save it securely.

![Console](./figures/console.upstage.ai.jpg)

In [2]:
# @title set API key
import os
import getpass

from pprint import pprint

import warnings

warnings.filterwarnings("ignore")

if "google.colab" in str(get_ipython()):
    # Running in Google Colab. Please set the UPSTAGE_API_KEY in the Colab Secrets
    from google.colab import userdata

    os.environ["UPSTAGE_API_KEY"] = userdata.get("UPSTAGE_API_KEY")
else:
    # Running locally. Please set the UPSTAGE_API_KEY in the .env file
    from dotenv import load_dotenv

    load_dotenv()

if "UPSTAGE_API_KEY" not in os.environ:
    os.environ["UPSTAGE_API_KEY"] = getpass.getpass("Enter your Upstage API key: ")

## 01. Auto Marketing
Using product descriptions and corresponding webpage content, Solar will create marketing email messages automatically.

In [3]:
# We will use FireCrawl, https://python.langchain.com/docs/integrations/document_loaders/firecrawl/
# You will need to set FIRECRAWL_API_KEY. Learn mode https://firecrawl.dev/.


if "FIRECRAWL_API_KEY" not in os.environ:
    os.environ["FIRECRAWL_API_KEY"] = getpass.getpass("Enter your Firecrawl API key: ")

Enter your Firecrawl API key:  ········


In [4]:
product_info = """
UpstageAI provides AI-driven tools and solutions aimed at enhancing productivity, automating tasks, and assisting decision-making for businesses. Their core offerings include:

1. **Solar Pro Preview**: A cutting-edge, highly intelligent large language model (LLM) that runs efficiently on a single GPU, designed to support complex tasks like data analysis and decision support. It's particularly suitable for businesses seeking advanced AI capabilities.
   
2. **Solar Mini**: A purpose-trained LLM tailored for specific tasks, providing businesses with powerful yet efficient AI functionalities.
   
3. **Document AI Tools**: These include Document Parsing, OCR (Optical Character Recognition), and Key Information Extraction, all designed to streamline document handling. Users can extract tables, figures, and text while enabling automation workflows for data management.

4. **Task APIs**: Upstage offers several APIs, such as:
   - **Chat**: For building conversational agents.
   - **Translation**: Context-aware English-Korean translation.
   - **Embeddings**: For text-to-vector conversion.
   - **Groundedness Check**: Ensures AI-generated responses are based on accurate data.
   - **Text-to-SQL**: Converts natural language into SQL queries (coming soon).

5. **Industry-Specific Intelligence**: Upstage is developing specialized APIs for industries such as healthcare, finance, and law, providing domain-specific AI solutions (coming soon).

Upstage focuses on automating repetitive tasks, improving business decision-making, and providing generative business intelligence tools to increase efficiency across various sectors.
"""

In [5]:
#(Scraping) Extract web page content from URLs and convert it to Markdown or structured data. This data is provided in a format that is easy to integrate with LLM.
from langchain_community.document_loaders import FireCrawlLoader

target_page = "https://www.firecrawl.dev/"
loader = FireCrawlLoader(url=target_page, mode="scrape")
docs = loader.load()

In [6]:
pprint(docs[0].metadata)

{'description': 'Firecrawl crawls and converts any website into clean '
                'markdown.',
 'keywords': 'Firecrawl,Markdown,Data,Mendable,Langchain',
 'language': 'en',
 'ogDescription': 'Turn any website into LLM-ready data.',
 'ogImage': 'https://www.firecrawl.dev/og.png?123',
 'ogLocaleAlternate': [],
 'ogSiteName': 'Firecrawl',
 'ogTitle': 'Firecrawl',
 'ogUrl': 'https://www.firecrawl.dev/',
 'pageStatusCode': 200,
 'robots': 'follow, index',
 'sitemap': {'changefreq': 'weekly', 'lastmod': '2024-09-24T04:10:15.767Z'},
 'sourceURL': 'https://www.firecrawl.dev/',
 'title': 'Home - Firecrawl'}


In [7]:
import marko
from marko.inline import RawText, Emphasis, CodeSpan, Link, Image
import re


def extract_clean_text(markdown_text):
    # Parse the markdown into an AST
    ast = marko.parse(markdown_text)

    clean_text = []

    def traverse(node):
        # If the node is a RawText, append its content
        if isinstance(node, RawText):
            clean_text.append(node.children)
        # If the node has children, recurse into them unless it's a link or image
        elif hasattr(node, "children") and not isinstance(node, (Link, Image)):
            for child in node.children:
                traverse(child)
        # For other node types that contain text (e.g., Emphasis, CodeSpan), handle accordingly
        elif isinstance(node, (Emphasis, CodeSpan)):
            traverse(node.children if hasattr(node, "children") else node)
        # You can add more conditions here if you want to handle other specific node types

    # Start traversing from the root of the AST
    traverse(ast)

    # Join all extracted text parts with spaces and clean up whitespace
    text = " ".join(clean_text).strip()

    # Remove URLs
    text = re.sub(r"http[s]?://\S+", "", text)
    text = re.sub(r"\bwww\.\S+", "", text)

    return text


# Use the function to extract clean text
target_info = extract_clean_text(docs[0].page_content)

# Print the cleaned text
pprint(target_info[:100])

('Our first Launch Week is over!  Beta Features Resources     14.7k Turn '
 'websites into    LLM-ready  d')


In [8]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_upstage import ChatUpstage

marketing_prompt_teample = ChatPromptTemplate.from_messages(
    [
        (
            "human",
            """You are a marketing specialist at UpsatgeAI selling the product.
Write a convincing and professional email message to a target company to sell your product.
Use the provided product information and details about the target company to personalize the message effectively.
Make your message relevant to the target company.

---
**Requirements:**
- The email should have a clear subject line.
- Highlight how your product can solve specific problems or add value to the target company.
- Maintain a professional and engaging tone.
- Include a call to action encouraging the recipient to take the next step.

---  
**Target Company Information to Email:** 
{target_company_info}
---   
**Your Product to Sell:** 
{product_info}

""",
        )
    ]
)

In [9]:
# write prompt
prompt = marketing_prompt_teample.format_messages(
    target_company_info=target_info,
    product_info=product_info,
)

print(prompt[0].content)

You are a marketing specialist at UpsatgeAI selling the product.
Write a convincing and professional email message to a target company to sell your product.
Use the provided product information and details about the target company to personalize the message effectively.
Make your message relevant to the target company.

---
**Requirements:**
- The email should have a clear subject line.
- Highlight how your product can solve specific problems or add value to the target company.
- Maintain a professional and engaging tone.
- Include a call to action encouraging the recipient to take the next step.

---  
**Target Company Information to Email:** 
Our first Launch Week is over!  Beta Features Resources     14.7k Turn websites into    LLM-ready  data Power your AI apps with clean data crawled from any website. It's also open-source. Start for free (500 credits)Start for free  A product by Crawl, Scrape, Clean We crawl all accessible subpages and give you clean markdown for each. No sitemap

In [10]:
#'solar-pro' is another option for the model. As of September 2024, the model is in preview status, with the final model expected to be released in November.
llm = ChatUpstage(model="solar-1-mini-chat")
chain = marketing_prompt_teample | llm | StrOutputParser()

email_msg = chain.invoke(
    {
        "target_company_info": target_info,
        "product_info": product_info,
    }
)

print(email_msg)

Subject: Enhance Your AI Capabilities with UpstageAI

Dear [Target Company Contact],

I hope this email finds you well. I am reaching out to you today to introduce you to UpstageAI, a cutting-edge AI solutions provider that specializes in enhancing productivity, automating tasks, and assisting decision-making for businesses.

I came across your company's website and noticed that you are currently using Firecrawl for your web scraping and crawling needs. While Firecrawl is a great tool for extracting data from websites, I believe that UpstageAI's offerings can significantly improve your AI capabilities and provide you with a more comprehensive solution.

UpstageAI provides a range of AI-driven tools and solutions, including:

1. Solar Pro Preview: A highly intelligent large language model (LLM) that runs efficiently on a single GPU, designed for complex tasks like data analysis and decision support.
2. Solar Mini: A purpose-trained LLM tailored for specific tasks, offering powerful yet 

## 02. Auto Filling
Based on your information, Solar will fill out the form automatically.

<a target="_blank" href="https://colab.research.google.com/github/UpstageAI/cookbook/blob/main/Solar-Fullstack-LLM-101/16_Reasoning.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [11]:
! pip3 install -qU langchain langchain-upstage langchain_community python-dotenv

## UPSTAGE_API_KEY
To obtain your Upstage API key, follow these steps:

1. Visit the Upstage AI console at <https://console.upstage.ai>.
2. Sign up for an account if you don't already have one.
3. Log in to your account.
4. Navigate to the API key section.
5. Generate your API key.
6. Copy the key and save it securely.

![Console](./figures/console.upstage.ai.jpg)

In [12]:
# @title set API key
import os
import getpass

from pprint import pprint

import warnings

warnings.filterwarnings("ignore")

if "google.colab" in str(get_ipython()):
    # Running in Google Colab. Please set the UPSTAGE_API_KEY in the Colab Secrets
    from google.colab import userdata

    os.environ["UPSTAGE_API_KEY"] = userdata.get("UPSTAGE_API_KEY")
else:
    # Running locally. Please set the UPSTAGE_API_KEY in the .env file
    from dotenv import load_dotenv

    load_dotenv()

if "UPSTAGE_API_KEY" not in os.environ:
    os.environ["UPSTAGE_API_KEY"] = getpass.getpass("Enter your Upstage API key: ")

In [13]:
my_info = """
John Do, born on April 15, 1985, is a Korean national and currently the CEO of UpstageAI, an AI-driven company based in the United States. He lives in Springfield, IL, at 1234 Maple Street, with his wife Julie Do and their two children, John Do Jr. and Jane Do. Both John and his wife, also born on April 15, 1985, hold Korean passports with expiration dates on September 30, 2025. The family, rooted in their Korean heritage, is well-supported by Korean national health insurance and enjoys strong ties to both Korea and the United States.

Professionally, John earned his Bachelor’s degree in Computer Science from Crestwood University (2004-2008) and an MBA from Brookstone Business School (2010-2012). His career journey saw him start as a Software Developer at InnovateX, where he worked on both front-end and back-end projects. He then moved on to become a Senior Software Engineer at TechCorp, where he led a team of engineers. In 2018, John became the CEO of UpstageAI, where he oversaw its remarkable 150% revenue growth within two years and launched several successful AI products used by Fortune 500 companies.

John is an active member of the international tech community, speaking at conferences and publishing articles in AI journals. He travels frequently, particularly to Europe, where he enjoys annual vacations lasting a week or two. Photography, hiking, reading science fiction novels, and playing the guitar are some of his hobbies. John is also an advocate for emerging technologies, including blockchain and quantum computing, and mentors young entrepreneurs.

In his personal life, John and Julie are dedicated parents to their children, who were born on April 15, 2010, and April 15, 2012. His emergency contact is Julie Do, and they can be reached at sung.kim@example.com or (555) 123-4567. They usually stay at hotels during their European trips, and the purpose of their visits is leisure.

John’s favorite quote is, “The best way to predict the future is to invent it,” by Alan Kay, which he lives by both in his career and personal life.
"""

In [14]:
# France Visit Application Form Fields

form_fields = [
    {
        "Field Name": "Full Name",
        "Description": "Applicant's complete legal name as shown on the passport."
    },
    {
        "Field Name": "Date of Birth",
        "Description": "Applicant's date of birth (DD/MM/YYYY)."
    },
    {
        "Field Name": "Nationality",
        "Description": "Applicant's current nationality."
    },
    {
        "Field Name": "Passport Number",
        "Description": "Unique number of the applicant's passport."
    },
    {
        "Field Name": "Passport Expiry Date",
        "Description": "Expiration date of the passport (DD/MM/YYYY)."
    },
    {
        "Field Name": "Purpose of Visit",
        "Description": "Reason for traveling to France (e.g., tourism, business, study)."
    },
    {
        "Field Name": "Duration of Stay",
        "Description": "Planned length of stay in France (e.g., 2 weeks, 3 months)."
    },
    {
        "Field Name": "Accommodation Details",
        "Description": "Information about where the applicant will be staying in France."
    },
    {
        "Field Name": "Financial Means",
        "Description": "Proof of sufficient funds to cover the stay (e.g., bank statements)."
    },
    {
        "Field Name": "Employment Status",
        "Description": "Current employment status and occupation of the applicant."
    },
    {
        "Field Name": "Travel Insurance",
        "Description": "Details of travel insurance covering medical expenses and emergencies."
    },
    {
        "Field Name": "Previous Visits to France",
        "Description": "Information about any prior visits to France, including dates and purposes."
    },
    {
        "Field Name": "Criminal Record",
        "Description": "Disclosure of any criminal convictions or legal issues."
    },
    {
        "Field Name": "Emergency Contact",
        "Description": "Contact information of a person to be reached in case of emergency."
    },
    {
        "Field Name": "Supporting Documents",
        "Description": "List of additional documents required to support the application (e.g., invitation letter, itinerary)."
    },
    {
        "Field Name": "Travel Itinerary",
        "Description": "Detailed schedule of travel plans while in France."
    },
    {
        "Field Name": "Proof of Accommodation",
        "Description": "Documents confirming hotel reservations or invitation from a host in France."
    },
    {
        "Field Name": "Visa Type",
        "Description": "Specific type of visa being applied for (e.g., short-stay, long-stay)."
    },
    {
        "Field Name": "Health Insurance",
        "Description": "Proof of health insurance coverage during the stay in France."
    },
    {
        "Field Name": "Marital Status",
        "Description": "Applicant's current marital status."
    },
    {
        "Field Name": "Dependents",
        "Description": "Information about any dependents traveling with the applicant."
    },
    {
        "Field Name": "Address in Home Country",
        "Description": "Applicant's residential address in their home country."
    },
    {
        "Field Name": "Contact Information",
        "Description": "Applicant's phone number and email address."
    },
    {
        "Field Name": "Language Proficiency",
        "Description": "Applicant's proficiency in French or other languages."
    }
]

In [15]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_upstage import ChatUpstage

form_filling_prompt_teample = ChatPromptTemplate.from_messages(
    [
        (
            "human",
            """You are a form filler. Fill out the form based on the information below.
            If you can guess from personal info, fill it out and make "[GUESS]".
            Do your best to fill out the form.

            If you don't know and no way to guess, just leave it and provide the reason why you left it blank.

            Write the filled form in JSON format.

---  
**Your Information:** 
{my_info}
---   
**Form to Fill:** 
{form_fields}
""",
        )
    ]
)

In [16]:
# write prompt
prompt = form_filling_prompt_teample.format_messages(
    my_info=my_info,
    form_fields=form_fields,
)

print(prompt[0].content)

You are a form filler. Fill out the form based on the information below.
            If you can guess from personal info, fill it out and make "[GUESS]".
            Do your best to fill out the form.

            If you don't know and no way to guess, just leave it and provide the reason why you left it blank.

            Write the filled form in JSON format.

---  
**Your Information:** 

John Do, born on April 15, 1985, is a Korean national and currently the CEO of UpstageAI, an AI-driven company based in the United States. He lives in Springfield, IL, at 1234 Maple Street, with his wife Julie Do and their two children, John Do Jr. and Jane Do. Both John and his wife, also born on April 15, 1985, hold Korean passports with expiration dates on September 30, 2025. The family, rooted in their Korean heritage, is well-supported by Korean national health insurance and enjoys strong ties to both Korea and the United States.

Professionally, John earned his Bachelor’s degree in Computer S

In [17]:
#'solar-pro' is another option for the model. As of September 2024, the model is in preview status, with the final model expected to be released in November.
solar_pro = ChatUpstage(model="solar-pro")
solar_mini = ChatUpstage(model="solar-1-mini-chat")

chain_pro = form_filling_prompt_teample | solar_pro | StrOutputParser()
chain_mini = form_filling_prompt_teample | solar_mini | StrOutputParser()


In [18]:

filled_form = chain_pro.invoke(
    {
        "my_info": my_info,
        "form_fields": form_fields,
    }
)

print(filled_form)


{
  "Full Name": "John Do",
  "Date of Birth": "[GUESS] 15/04/1985",
  "Nationality": "Korean",
  "Passport Number": "[LEAVE] Not provided",
  "Passport Expiry Date": "30/09/2025",
  "Purpose of Visit": "[GUESS] Leisure",
  "Duration of Stay": "[GUESS] 2 weeks",
  "Accommodation Details": "[GUESS] Hotel",
  "Financial Means": "[LEAVE] Not provided",
  "Employment Status": "CEO of UpstageAI",
  "Travel Insurance": "[LEAVE] Not provided",
  "Previous Visits to France": "[LEAVE] Not provided",
  "Criminal Record": "[LEAVE] Not provided",
  "Emergency Contact": "Julie Do, sung.kim@example.com, (555) 123-4567",
  "Supporting Documents": "[LEAVE] Not provided",
  "Travel Itinerary": "[LEAVE] Not provided",
  "Proof of Accommodation": "[LEAVE] Not provided",
  "Visa Type": "[GUESS] Short-stay",
  "Health Insurance": "[GUEST] Korean national health insurance",
  "Marital Status": "Married",
  "Dependents": "[LEAVE] Not provided",
  "Address in Home Country": "1234 Maple Street, Springfield, IL

In [21]:
university_application_form = [
    {
        "Field Name": "Full Name",
        "Description": "Applicant's complete legal name as shown on official documents."
    },
    {
        "Field Name": "Date of Birth",
        "Description": "Applicant's date of birth (DD/MM/YYYY)."
    },
    {
        "Field Name": "Gender",
        "Description": "Applicant's gender identity."
    },
    {
        "Field Name": "Nationality",
        "Description": "Applicant's current nationality."
    },
    {
        "Field Name": "Email Address",
        "Description": "Applicant's primary email address for correspondence."
    },
    {
        "Field Name": "Phone Number",
        "Description": "Applicant's contact phone number."
    },
    {
        "Field Name": "Mailing Address",
        "Description": "Applicant's current mailing address."
    },
    {
        "Field Name": "Intended Major",
        "Description": "The field of study the applicant wishes to pursue."
    },
    {
        "Field Name": "Degree Level",
        "Description": "The level of degree the applicant is applying for (e.g., Bachelor's, Master's, PhD)."
    },
    {
        "Field Name": "High School Name",
        "Description": "Name of the high school the applicant graduated from."
    },
    {
        "Field Name": "High School Graduation Date",
        "Description": "Date of high school graduation (MM/YYYY)."
    },
    {
        "Field Name": "GPA",
        "Description": "Applicant's Grade Point Average from their most recent academic institution."
    },
    {
        "Field Name": "Standardized Test Scores",
        "Description": "Scores from relevant standardized tests (e.g., SAT, ACT, GRE, TOEFL, IELTS)."
    },
    {
        "Field Name": "Extracurricular Activities",
        "Description": "List of the applicant's extracurricular activities, clubs, and organizations."
    },
    {
        "Field Name": "Work Experience",
        "Description": "Relevant work experience, internships, or volunteer positions."
    },
    {
        "Field Name": "Awards and Honors",
        "Description": "Any academic or extracurricular awards or honors received."
    },
    {
        "Field Name": "Personal Statement",
        "Description": "A brief essay about the applicant's background, goals, and reasons for applying."
    },
    {
        "Field Name": "References",
        "Description": "Names and contact information for academic or professional references."
    },
    {
        "Field Name": "Financial Aid",
        "Description": "Indication of whether the applicant is applying for financial aid or scholarships."
    },
    {
        "Field Name": "Language Proficiency",
        "Description": "Applicant's proficiency in the language of instruction and any other relevant languages."
    },
    {
        "Field Name": "Previous University Education",
        "Description": "Details of any previous university-level education, if applicable."
    },
    {
        "Field Name": "Emergency Contact",
        "Description": "Name and contact information for an emergency contact person."
    },
]

In [22]:
filled_form = chain_mini.invoke(
    {
        "my_info": my_info,
        "form_fields": university_application_form,
    }
)

print(filled_form)

{
  "Full Name": "John Do",
  "Date of Birth": "15/04/1985",
  "Gender": "[GUESS]",
  "Nationality": "Korean",
  "Email Address": "[GUESS]",
  "Phone Number": "[GUESS]",
  "Mailing Address": "1234 Maple Street, Springfield, IL",
  "Intended Major": "[GUESS]",
  "Degree Level": "[GUESS]",
  "High School Name": "[GUESS]",
  "High School Graduation Date": "[GUESS]",
  "GPA": "[GUESS]",
  "Standardized Test Scores": "[GUESS]",
  "Extracurricular Activities": "[GUESS]",
  "Work Experience": "[GUESS]",
  "Awards and Honors": "[GUESS]",
  "Personal Statement": "[GUESS]",
  "References": "[GUESS]",
  "Financial Aid": "[GUESS]",
  "Language Proficiency": "[GUESS]",
  "Previous University Education": "[GUESS]",
  "Emergency Contact": "Julie Do, sung.kim@example.com, (555) 123-4567"
}


In [23]:
filled_form = chain_pro.invoke(
    {
        "my_info": my_info,
        "form_fields": university_application_form,
    }
)

print(filled_form)

{
  "Full Name": "John Do",
  "Date of Birth": "15/04/1985",
  "Gender": "[GUESS]",
  "Nationality": "Korean",
  "Email Address": "sung.kim@example.com",
  "Phone Number": "(555) 123-4567",
  "Mailing Address": "1234 Maple Street, Springfield, IL",
  "Intended Major": "N/A",
  "Degree Level": "N/A",
  "High School Name": "N/A",
  "High School Graduation Date": "N/A",
  "GPA": "N/A",
  "Standardized Test Scores": "N/A",
  "Extracurricular Activities": "N/A",
  "Work Experience": "Software Developer at InnovateX, Senior Software Engineer at TechCorp, CEO of UpstageAI",
  "Awards and Honors": "N/A",
  "Personal Statement": "N/A",
  "References": "N/A",
  "Financial Aid": "N/A",
  "Language Proficiency": "N/A",
  "Previous University Education": "Bachelor’s degree in Computer Science from Crestwood University (2004-2008), MBA from Brookstone Business School (2010-2012)",
  "Emergency Contact": "Julie Do, sung.kim@example.com or (555) 123-4567"
}


In [24]:
company_leave_form = [
    {
        "Field Name": "Employee Name",
        "Description": "Full name of the employee requesting leave."
    },
    {
        "Field Name": "Employee ID",
        "Description": "Unique identification number assigned to the employee."
    },
    {
        "Field Name": "Department",
        "Description": "Department or division where the employee works."
    },
    {
        "Field Name": "Position",
        "Description": "Employee's current job title or position."
    },
    {
        "Field Name": "Leave Type",
        "Description": "Type of leave being requested (e.g., Annual, Sick, Maternity, Paternity, Bereavement, etc.)."
    },
    {
        "Field Name": "Reason for Leave",
        "Description": "Brief explanation for the leave request."
    },
    {
        "Field Name": "Contact During Leave",
        "Description": "Phone number or email where the employee can be reached during leave, if necessary."
    },
    {
        "Field Name": "Emergency Contact",
        "Description": "Name and contact information for an emergency contact person."
    },
   
]

In [25]:
filled_form = chain_mini.invoke(
    {
        "my_info": my_info,
        "form_fields": company_leave_form,
    }
)

print(filled_form)

{
  "Employee Name": "John Do",
  "Employee ID": "[GUESS]",
  "Department": "[GUESS]",
  "Position": "[GUESS]",
  "Leave Type": "[GUESS]",
  "Reason for Leave": "[GUESS]",
  "Contact During Leave": "[GUESS]",
  "Emergency Contact": "Julie Do, sung.kim@example.com, (555) 123-4567"
}


In [26]:
filled_form = chain_pro.invoke(
    {
        "my_info": my_info,
        "form_fields": company_leave_form,
    }
)

print(filled_form)

{
  "Employee Name": "John Do",
  "Employee ID": "[GUESS]",
  "Department": "CEO of UpstageAI",
  "Position": "CEO",
  "Leave Type": "N/A",
  "Reason for Leave": "N/A",
  "Contact During Leave": "N/A",
  "Emergency Contact": "Julie Do, (555) 123-4567, sung.kim@example.com"
}


## 03. Contract Advising
Solar will assist in analyzing and advising on contracts by retrieving relevant information and generating legal insights.

<a target="_blank" href="https://colab.research.google.com/github/UpstageAI/cookbook/blob/main/Solar-Fullstack-LLM-101/16_Reasoning.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Contract Advising


In [ ]:
! pip3 install -qU langchain langchain-upstage langchain_community python-dotenv

## UPSTAGE_API_KEY
To obtain your Upstage API key, follow these steps:

1. Visit the Upstage AI console at <https://console.upstage.ai>.
2. Sign up for an account if you don't already have one.
3. Log in to your account.
4. Navigate to the API key section.
5. Generate your API key.
6. Copy the key and save it securely.

![Console](./figures/console.upstage.ai.jpg)

In [ ]:
# @title set API key
import os
import getpass

from pprint import pprint

import warnings

warnings.filterwarnings("ignore")

if "google.colab" in str(get_ipython()):
    # Running in Google Colab. Please set the UPSTAGE_API_KEY in the Colab Secrets
    from google.colab import userdata

    os.environ["UPSTAGE_API_KEY"] = userdata.get("UPSTAGE_API_KEY")
else:
    # Running locally. Please set the UPSTAGE_API_KEY in the .env file
    from dotenv import load_dotenv

    load_dotenv()

if "UPSTAGE_API_KEY" not in os.environ:
    os.environ["UPSTAGE_API_KEY"] = getpass.getpass("Enter your Upstage API key: ")

In [27]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_upstage import ChatUpstage

contract_advising_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an expert contract advisor. 
Your task is to revise the given contract draft based on best practices in contract writing.

Follow these steps:

1. Review the provided contract advisory examples carefully.
2. Analyze the given contract draft.
3. Revise the contract draft to improve clarity, accuracy, and legal soundness.
4. Provide a clean, revised version of the entire contract.
5. After the revised contract, add an appendix that explains your key revisions and the reasons behind them.

Remember to:
- Use clear and concise language
- Correct any errors or ambiguities
- Ensure all parties and terms are clearly defined
- Add any necessary clauses that may be missing

---
Contract Advisory Examples:
{contact_advise_examples}
---
Original Contract Draft:
{contract_draft}
---
Revised Contract Draft:
""",
        )
    ]
)

In [28]:
#'solar-pro' is another option for the model. As of September 2024, the model is in preview status, with the final model expected to be released in November.
solar_pro = ChatUpstage(model="solar-pro")
solar_mini = ChatUpstage(model="solar-1-mini-chat")

chain_pro = contract_advising_prompt | solar_pro | StrOutputParser()
chain_mini = contract_advising_prompt | solar_mini | StrOutputParser()


In [31]:
# from https://www.linkedin.com/pulse/examples-bad-better-contract-drafting-richard-bennett/
contact_advise = """
Below are three examples of contract provisions that were in different “model form contracts” published by well-respected legal publishers. I’ve rewritten them to be more understandable and to correct key errors in them. I’ve italicized the key errors in each example for easy reference.

Example 1

Within seven (7) days after the signing of this Agreement, Buyer agrees to furnish Seller with each of the Seller’s bank statements over the past six (6) months.

Minimum revision needed:

Within seven (7) days after the signing of this Agreement, Buyer agrees to furnish Seller with each of the Buyer’s bank statements over the past six (6) months.

Better version:

Within 7 days after the delivery of the fully signed Agreement, Buyer will give Seller copies of Buyer’s last 6 months of its bank statements.

Even better version (depending on the circumstances):

No later than [date], Buyer will give Seller copies of Buyer’s bank statements for the six months ending [date].

Or if your client is Seller:

No later than [date], Seller will receive from Buyer copies of Buyer’s bank statements for the six months ending [date].

Example 2

If any litigation arises from or in connection with this Agreement, each party hereto agrees to pay the attorney’s fees of the other.

Minimum revision needed:

If any litigation arises from or in connection with this Agreement, the non-prevailing party hereto agrees to pay the attorney’s fees of the prevailing party.

Better version:

The prevailing party in any litigation related to this Agreement will be entitled to an award of its attorney’s fees.

Even better version (depending on the circumstances):

If a dispute related to this Agreement arises between the parties, the non-prevailing party will pay the prevailing party all reasonable attorney’s fees that the prevailing party incurs regardless of whether a lawsuit is initiated.

Note: The phrase “prevailing party” can be problematical in some jurisdictions.

Example 3

Contractor hereby agrees prior to commencement of the services contemplated hereunder that it shall furnish to Client satisfactory evidence of the insurance coverage required hereunder and the certificate shall provide a thirty (30) day written notification provision that notice to Client is required to be sent prior to cancellation of or material change to such insurance.

Minimum revision needed:

Contractor hereby agrees that it shall furnish to Client satisfactory evidence of the insurance coverage required hereunder prior to commencement of the services contemplated hereunder, and the certificate shall provide a thirty (30) day written notification provision that notice to Client is required to be sent prior to cancellation of or material change to such insurance.

Better revision:

Before the commencement of the services contemplated by this Agreement, Contractor will give Client an insurance certificate evidencing the insurance coverage required by this Agreement. No such insurance may be cancelled or materially changed without at least 30 days written notice to Client, and the insurance certificate must reflect such provisions.

Even better revision:

Before the Contractor begins providing services under this Agreement, Contractor must give Client either (i) copies of all insurance policies and endorsements for each type of insurance coverage required by this Agreement or (ii) an insurance binder plus proof of payment of all premiums required (or proof that no additional premium is required) for each type of insurance coverage required by this Agreement. No such insurance may be cancelled or materially changed without at least 30 days written notice to Client, and the foregoing documents that Contractor provides must reflect such notice provisions.

Lesson One — Don’t rely on form contracts. Read them. They may contain critical errors.

Lesson Two — Clear writing is important. If the contract you are working with has unclear language, fix it. It should be readable and understandable by anyone. And it should accurately reflect your intent.
"""

In [32]:
contract_draft = """
CONTRACT AND BILL OF SALE FOR AUTOMOBILE
("Seller") agrees to sell to
("Buyer")
the following automobile: [specify make, model and year of car], with Vehicle Identification (the "Vehicle"), pursuant to these terms:
1. The date of the sale of the Vehicle will be
, 20_
2. The total purchase price of the Vehicle will be
Dollars
($
3. In exchange for the Vehicle, the Buyer will pay Seller the total purchase price of the Vehicle on the day of the sale by cashier's check, money order, or cash. [ALTERNATIVE:
In exchange for Vehicle, on the day of the sale, Buyer will pay Seller $ cashier's check, money order, or cash as a downpayment for the vehicle and will sign a promissory note for the balance of the total purchase price, which shall be due in full on or before
•]
4. Upon receipt of payment as provided above, The Seller agrees to provide the following documents to Buyer on the sale date:
: Che terret registration for the Vehite.
Certificate of Title (including Odometer Disclosure Section), signed by Seller.
5. The Seller agrees to deliver the Vehicle to Buyer with a current registration and a clear title. Seller warrants that Seller is the legal owner of the Vehicle and that the Vehicle is free of all legal claims, liens, and encumbrances. The Seller agrees to pay for and deliver any necessary smog certification to Buyer before the sale date. [NOTE: an updated smog certification may not be required if the Vehicle is less than four years old.]
6. The Vehicle is sold "as is," and the Seller makes no express or implied warranties as to the condition or performance of the Vehicle, except as follows: to the best of Seller's knowledge, this vehicle:
• is is not a salvage vehicle.
• has has not been declared a total loss by an insurance company.
• has has not been repaired pursuant to a Lemon Law.
7. The Buyer agrees to register the Vehicle in his/her name with the California Department of Motor Vehicles within one week of the date of the sale.
Date:
[Name of Seller ]
Date:
[Name of Buyer]
*OTHER POINTS TO REMEMBER:
• The Seller should file a Notice of Transfer and Release of Liability with the DMV within 5 days of the sale.
• The Buyer must go to the DMV to compete the transfer of ownership within 10 days of the sale.
"""


In [33]:
revised_contract = chain_mini.invoke(
    {
        "contact_advise_examples": contact_advise,
        "contract_draft": contract_draft,
    }
)

print(revised_contract)

CONTRACT AND BILL OF SALE FOR AUTOMOBILE

This Contract and Bill of Sale (the "Agreement") is made and entered into on the date of execution (the "Effective Date") by and between [Seller's Name] (the "Seller") and [Buyer's Name] (the "Buyer").

1. Sale of Vehicle. Seller agrees to sell to Buyer the following automobile: [specify make, model, and year of car], with Vehicle Identification Number [insert VIN], (the "Vehicle"), pursuant to the terms and conditions set forth in this Agreement.
2. Sale Date. The date of the sale of the Vehicle shall be [insert date].
3. Purchase Price. The total purchase price of the Vehicle shall be $[insert amount] (the "Purchase Price").
4. Payment. In exchange for the Vehicle, Buyer shall pay Seller the Purchase Price on the date of the sale by cashier's check, money order, or cash. [ALTERNATIVE: In exchange for the Vehicle, on the date of the sale, Buyer will pay Seller $[insert down payment amount] by cashier's check, money order, or cash as a down pay

In [34]:
revised_contract = chain_pro.invoke(
    {
        "contact_advise_examples": contact_advise,
        "contract_draft": contract_draft,
    }
)

print(revised_contract)

Revised Contract and Bill of Sale for Automobile

("Seller") agrees to sell to ("Buyer") the following automobile: [specify make, model, and year of car], with Vehicle Identification (the "Vehicle"), pursuant to these terms:

1. The date of the sale of the Vehicle will be [date], 20_
2. The total purchase price of the Vehicle will be $[amount] Dollars ($[amount]).
3. In exchange for the Vehicle, the Buyer will pay Seller the total purchase price of the Vehicle on the day of the sale by cashier's check, money order, or cash.
4. Upon receipt of payment as provided above, the Seller agrees to provide the following documents to Buyer on the sale date:
   a. The current registration for the Vehicle.
   b. The Certificate of Title (including Odometer Disclosure Section), signed by Seller.
5. The Seller agrees to deliver the Vehicle to Buyer with a current registration and a clear title. Seller warrants that Seller is the legal owner of the Vehicle and that the Vehicle is free of all legal cl